In [ ]:
# Cell 1: Installera och importera nödvändiga bibliotek
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# MAX78002 specific imports
import max78_modules.ai8x as ai8x
import distiller

# Progress bar
from tqdm import tqdm

print("Bibliotek importerade!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA tillgänglig: {torch.cuda.is_available()}")


In [ ]:
# Cell 2: Setup MAX78002 device

class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

args = Args(act_mode_8bit=False)
ai8x.set_device(87, simulate=False, round_avg=False)  # 87 for MAX78002
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"MAX78002 device configured")
print(f"Training device: {device}")


In [ ]:
# Cell 3: Load data from MIT-BIH physionet dataset
from ecg_dataset import MITBIHDataset

ecg_train_raw = MITBIHDataset(
    root_dir='./physionet.org/files/mitdb/1.0.0',
    train=True,
    window_size=128,
    transform=None,
    oversample_minority=False  # Using natural imbalanced data
)

ecg_test = MITBIHDataset(
    root_dir='./physionet.org/files/mitdb/1.0.0',
    train=False,
    window_size=128,
    transform=None,
    oversample_minority=False
)

class_names = {i: name for i, name in enumerate(ecg_train_raw.class_names)}
print(f"Training data shape: {ecg_train_raw.data.shape}")
print(f"Test data shape: {ecg_test.data.shape}")

# Use original data without SMOTE
X_train = ecg_train_raw.data
y_train = ecg_train_raw.labels
X_test = ecg_test.data
y_test = ecg_test.labels

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"\nOriginal class distribution:")
print(Counter(y_train))

In [ ]:
# Cell 5: Create ai8x-compatible datasets
from torchvision import transforms

# Create a proper get_datasets function following ai8x pattern
def ecg_mitbih_get_datasets(data, load_train=True, load_test=True):
    """
    Load ECG MIT-BIH dataset with ai8x compatibility
    """
    (data_dir, args) = data
    
    # For 1D data, we only need ai8x normalization (no ToTensor needed)
    transform = ai8x.normalize(args=args)
    
    train_dataset = None
    test_dataset = None
    
    if load_train:
        # Wrap the balanced training data
        class ECGTrainDataset(Dataset):
            def __init__(self, X, y, transform=None):
                self.X = X
                self.y = y
                self.transform = transform
            
            def __len__(self):
                return len(self.y)
            
            def __getitem__(self, idx):
                # Shape: (128,) -> (1, 128) for single channel
                x = torch.FloatTensor(self.X[idx]).unsqueeze(0)
                if self.transform:
                    x = self.transform(x)
                y = torch.LongTensor([self.y[idx]]).squeeze()
                return x, y
        
        train_dataset = ECGTrainDataset(X_train, y_train, transform=transform)
    
    if load_test:
        # Wrap the test data
        class ECGTestDataset(Dataset):
            def __init__(self, X, y, transform=None):
                self.X = X
                self.y = y
                self.transform = transform
            
            def __len__(self):
                return len(self.y)
            
            def __getitem__(self, idx):
                # Shape: (128,) -> (1, 128) for single channel
                x = torch.FloatTensor(self.X[idx]).unsqueeze(0)
                if self.transform:
                    x = self.transform(x)
                y = torch.LongTensor([self.y[idx]]).squeeze()
                return x, y
        
        test_dataset = ECGTestDataset(X_test, y_test, transform=transform)
    
    return train_dataset, test_dataset


# Create datasets using ai8x pattern
train_dataset, test_dataset = ecg_mitbih_get_datasets(
    ('./data', args),
    load_train=True, 
    load_test=True
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=False)

print(f"Träningsbatchar: {len(train_loader)}")
print(f"Testbatchar: {len(test_loader)}")
print(f"Batch shape exempel: {next(iter(train_loader))[0].shape}")
print("\n✓ Datasets are now ai8x-compatible!")


In [ ]:
# Cell 6: Definiera ai8x-kompatibel modell för MAX78002
class ECGClassifier(nn.Module):
    """
    ai8x-compatible 1D CNN for ECG classification
    Designed for MAX78002 hardware constraints:
    - MAX78002 only allows padding=[0,1,2] for Conv1d
    - MAX78002 requires padding=0 when input channels > 64
    """
    def __init__(self, num_classes=5, num_channels=1, dimensions=(128, 1), bias=True, **kwargs):
        super(ECGClassifier, self).__init__()
        
        # Block 1: 64 filters (can use padding since <=64 channels)
        self.conv1_1 = ai8x.FusedConv1dBNReLU(num_channels, 64, kernel_size=5, padding=2, 
                                               bias=bias, **kwargs)
        self.conv1_2 = ai8x.FusedMaxPoolConv1dBNReLU(64, 64, kernel_size=5, padding=2, 
                                                      pool_size=2, pool_stride=2,
                                                      bias=bias, **kwargs)
        # (1, 128) -> (64, 128) -> (64, 64)
        
        # Block 2: 128 filters (MUST use padding=0 since >64 input channels)
        self.conv2_1 = ai8x.FusedConv1dBNReLU(64, 128, kernel_size=5, padding=2, 
                                               bias=bias, **kwargs)
        self.conv2_2 = ai8x.FusedMaxPoolConv1dBNReLU(128, 128, kernel_size=5, padding=0,
                                                      pool_size=2, pool_stride=2,
                                                      bias=bias, **kwargs)
        # (64, 64) -> (128, 64) -> (128, 30)
        
        # Block 3: 256 filters (padding=0 required)
        self.conv3_1 = ai8x.FusedConv1dBNReLU(128, 256, kernel_size=3, padding=0, 
                                               bias=bias, **kwargs)
        self.conv3_2 = ai8x.FusedMaxPoolConv1dBNReLU(256, 256, kernel_size=3, padding=0,
                                                      pool_size=2, pool_stride=2,
                                                      bias=bias, **kwargs)
        # (128, 30) -> (256, 28) -> (256, 13)
        
        # Block 4: 512 filters (padding=0 required)
        self.conv4_1 = ai8x.FusedMaxPoolConv1dBNReLU(256, 512, kernel_size=3, padding=0,
                                                      pool_size=2, pool_stride=2,
                                                      bias=bias, **kwargs)
        # (256, 13) -> (512, 5)
        
        # Final conv (padding=0 required)
        self.conv5 = ai8x.FusedConv1dBNReLU(512, 256, kernel_size=3, padding=0,
                                             bias=bias, **kwargs)
        # (512, 5) -> (256, 3)
        
        # Fully connected layer
        # After convolutions: (batch, 256, 1) = 256 features
        self.fc = ai8x.Linear(256 * 1, num_classes, bias=True, wide=True, **kwargs)
    
    def forward(self, x):
        # Block 1
        x = self.conv1_1(x)  # (batch, 64, 128)
        x = self.conv1_2(x)  # (batch, 64, 64)
        
        # Block 2
        x = self.conv2_1(x)  # (batch, 128, 64)
        x = self.conv2_2(x)  # (batch, 128, 30)
        
        # Block 3
        x = self.conv3_1(x)  # (batch, 256, 28)
        x = self.conv3_2(x)  # (batch, 256, 13)
        
        # Block 4
        x = self.conv4_1(x)  # (batch, 512, 5)
        
        # Final conv
        x = self.conv5(x)    # (batch, 256, 1)
        
        # Flatten for FC layer - use flatten instead of view for safety
        x = x.flatten(start_dim=1)  # (batch, 256)
        
        # Final classification
        x = self.fc(x)
        
        return x


# Skapa modell och flytta till GPU om tillgänglig
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ECGClassifier(num_classes=5).to(device)

print(f"Modell skapad på device: {device}")
if device.type == 'cpu':
    torch.set_num_threads(torch.get_num_threads())
    print(f"CPU-läge: Använder {torch.get_num_threads()} trådar")
print(f"\nAntal parametrar: {sum(p.numel() for p in model.parameters()):,}")
print(f"Träningsbara parametrar: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Debug: Test forward pass to verify shape
print("\n=== Shape Verification ===")
test_input = torch.randn(1, 1, 128).to(device)
model.eval()  # Set to eval mode to avoid BatchNorm issues with single sample
with torch.no_grad():
    test_output = model(test_input)
model.train()  # Set back to train mode
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {test_output.shape}")
print(f"Expected output shape: (1, 5)")
print("\n✓ Modellen är nu ai8x-kompatibel för MAX78002!")


In [ ]:
# Cell 7: Visualisera klassfördelning
class_names = {
    0: 'Normal (N)',
    1: 'Supraventricular (S)',
    2: 'Ventricular (V)',
    3: 'Fusion (F)',
    4: 'Unknown (Q)'
}

plt.figure(figsize=(12, 5))

# Training data
plt.subplot(1, 2, 1)
train_counts = pd.Series(y_train).value_counts().sort_index()
bars1 = plt.bar(train_counts.index, train_counts.values, color='steelblue', alpha=0.8)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Number of Samples', fontsize=12)
plt.title('Training Data Distribution (Imbalanced)', fontsize=14, fontweight='bold')
plt.xticks(range(5), [class_names[i] for i in range(5)], rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

for bar in bars1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom', fontsize=9)

# Test data
plt.subplot(1, 2, 2)
test_counts = pd.Series(y_test).value_counts().sort_index()
bars2 = plt.bar(test_counts.index, test_counts.values, color='coral', alpha=0.8)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Number of Samples', fontsize=12)
plt.title('Test Data Distribution', fontsize=14, fontweight='bold')
plt.xticks(range(5), [class_names[i] for i in range(5)], rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nNote: We're using class weights in the loss function to handle imbalance.")
print("This is better than SMOTE for time-series ECG data.")


In [ ]:
# Cell 8: Define Loss Function - NO WEIGHTS (Baseline Reference)
"""
Standard Cross-Entropy Loss WITHOUT class weights - for baseline reference.

What it does:
- Measures the difference between predicted class probabilities and true labels
- Combines log-softmax and negative log-likelihood in one operation
- Automatically applies softmax to model outputs, so the model doesn't need a softmax layer
- NO class balancing - lets the model learn naturally from imbalanced data
- This gives us a clean baseline to compare against other approaches

ai8x-training uses nn.CrossEntropyLoss() as the default criterion for classification tasks.
This is the recommended loss function for MAX78002 training workflows.
"""

print("Training data class distribution:")
for i, (name, count) in enumerate(sorted(Counter(y_train).items())):
    pct = (count / len(y_train)) * 100
    print(f"  {class_names[i]:20s}: {count:6,} samples ({pct:5.2f}%)")

# Create standard loss criterion WITHOUT weights
criterion = nn.CrossEntropyLoss().to(device)

print("\n✓ Loss function configured: CrossEntropyLoss (NO WEIGHTS)")
print(f"  - Device: {device}")
print(f"  - Compatible with ai8x-training framework")
print(f"  - Baseline reference - natural learning from imbalanced data")
print(f"  - Expected: High accuracy on Normal, lower on rare classes")

In [ ]:
# Cell 9: ai8x-compatible Training Functions with Compression Scheduler
"""
Training functions compatible with ai8x-training framework.
Integrates with Distiller compression scheduler for proper quantization workflow.
"""
import torchnet as tnt
from collections import OrderedDict

def train_epoch(model, dataloader, criterion, optimizer, device, epoch, 
                compression_scheduler=None, logger=None):
    """
    ai8x-compatible training loop for one epoch
    Follows the pattern from ai8x-training/train.py
    """
    # Use torchnet meters like ai8x-training does
    losses = OrderedDict([
        ('Overall Loss', tnt.meter.AverageValueMeter()),
        ('Objective Loss', tnt.meter.AverageValueMeter())
    ])
    classerr = tnt.meter.ClassErrorMeter(accuracy=True, topk=(1, 5))
    batch_time = tnt.meter.AverageValueMeter()
    
    total_samples = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    steps_per_epoch = len(dataloader)
    
    # Compression scheduler epoch begin callback
    if compression_scheduler:
        compression_scheduler.on_epoch_begin(epoch)
    
    # Switch to train mode
    model.train()
    
    # Progress bar for notebook visibility
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [Training]', leave=True, position=0)
    
    import time
    end = time.time()
    
    for train_step, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Compression scheduler minibatch begin callback
        if compression_scheduler:
            compression_scheduler.on_minibatch_begin(epoch, train_step, steps_per_epoch, optimizer)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Record loss
        losses['Objective Loss'].add(loss.item())
        
        if compression_scheduler:
            # Before running backward phase, allow scheduler to modify loss
            # (e.g. add regularization loss)
            agg_loss = compression_scheduler.before_backward_pass(
                epoch, train_step, steps_per_epoch, loss,
                optimizer=optimizer, return_loss_components=True
            )
            loss = agg_loss.overall_loss
            losses['Overall Loss'].add(loss.item())
            
            # Add any additional loss components from compression
            for lc in agg_loss.loss_components:
                if lc.name not in losses:
                    losses[lc.name] = tnt.meter.AverageValueMeter()
                losses[lc.name].add(lc.value.item())
        else:
            losses['Overall Loss'].add(loss.item())
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        
        # Compression scheduler before parameter optimization callback
        if compression_scheduler:
            compression_scheduler.before_parameter_optimization(epoch, train_step, steps_per_epoch, optimizer)
        
        optimizer.step()
        
        # Compression scheduler minibatch end callback
        if compression_scheduler:
            compression_scheduler.on_minibatch_end(epoch, train_step, steps_per_epoch)
        
        # Measure accuracy and record statistics
        classerr.add(outputs.data, targets)
        batch_time.add(time.time() - end)
        end = time.time()
        
        # Update progress bar
        top1 = classerr.value(1)
        pbar.set_postfix({
            'loss': f'{losses["Overall Loss"].mean:.4f}',
            'top1': f'{top1:.2f}%'
        })
    
    # Compression scheduler epoch end callback
    if compression_scheduler:
        compression_scheduler.on_epoch_end(epoch, optimizer)
    
    epoch_loss = losses['Overall Loss'].mean
    epoch_top1 = classerr.value(1)
    epoch_top5 = classerr.value(5)
    
    if logger:
        logger(f'==> Training Epoch {epoch}: Top1: {epoch_top1:.3f}  Top5: {epoch_top5:.3f}  '
               f'Loss: {epoch_loss:.3f}')
    
    return epoch_loss, epoch_top1, epoch_top5


def validate(model, dataloader, criterion, device, epoch, logger=None):
    """
    ai8x-compatible validation loop
    Follows the pattern from ai8x-training/train.py
    """
    # Use torchnet meters like ai8x-training does
    losses = OrderedDict([
        ('Overall Loss', tnt.meter.AverageValueMeter()),
        ('Objective Loss', tnt.meter.AverageValueMeter())
    ])
    classerr = tnt.meter.ClassErrorMeter(accuracy=True, topk=(1, 5))
    batch_time = tnt.meter.AverageValueMeter()
    
    total_samples = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    
    # Switch to evaluation mode
    model.eval()
    
    # Progress bar for notebook visibility
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [Validation]', leave=True, position=0)
    
    import time
    end = time.time()
    
    with torch.no_grad():
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Measure accuracy and record loss
            losses['Objective Loss'].add(loss.item())
            losses['Overall Loss'].add(loss.item())
            classerr.add(outputs.data, targets)
            
            # Measure elapsed time
            batch_time.add(time.time() - end)
            end = time.time()
            
            # Update progress bar
            top1 = classerr.value(1)
            pbar.set_postfix({
                'loss': f'{losses["Overall Loss"].mean:.4f}',
                'top1': f'{top1:.2f}%'
            })
    
    epoch_loss = losses['Overall Loss'].mean
    epoch_top1 = classerr.value(1)
    epoch_top5 = classerr.value(5)
    
    if logger:
        logger(f'==> Validation: Top1: {epoch_top1:.3f}  Top5: {epoch_top5:.3f}  '
               f'Loss: {epoch_loss:.3f}')
    
    return epoch_loss, epoch_top1, epoch_top5


# Simple logger function for notebook
def print_logger(msg):
    print(msg)

print("✓ ai8x-compatible training functions created!")
print("  - Integrates with Distiller compression scheduler")
print("  - Uses torchnet meters for metrics tracking")
print("  - Follows ai8x-training/train.py patterns")
print("  - Ready for quantization-aware training (QAT)")


In [ ]:
# Cell 10: Setup Optimizer and Compression Scheduler (ai8x-compatible)
"""
Following ai8x-training patterns for proper MAX78002 deployment:
- Adam optimizer with lr=0.001 (standard for ai8x)
- Distiller compression scheduler for pruning/QAT
- Proper learning rate scheduling
- Integration with ai8x-compatible training functions
"""

# Setup optimizer following ai8x patterns
# Adam with lr=0.001 is standard for most ai8x models
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0)

print("✓ Optimizer configured: Adam")
print(f"  - Learning rate: 0.001")
print(f"  - Weight decay: 0")
print(f"  - Compatible with ai8x-training framework")

# Split training data for validation (85/15 split is standard)
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(
    train_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

train_loader_split = DataLoader(train_subset, batch_size=64, shuffle=True, 
                                num_workers=4, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, 
                        num_workers=4, pin_memory=True)

print(f"\n✓ Data split:")
print(f"  - Training samples: {len(train_subset):,}")
print(f"  - Validation samples: {len(val_subset):,}")
print(f"  - Test samples: {len(test_dataset):,}")

# Setup Distiller compression scheduler
# This is ESSENTIAL for ai8x quantization and pruning
compression_scheduler = distiller.CompressionScheduler(model)

print("\n✓ Compression scheduler initialized")
print("  - Ready for pruning policies")
print("  - Ready for quantization-aware training (QAT)")
print("  - Integrates with training loop callbacks")


In [ ]:
# Cell 10.5: Enable Quantization-Aware Training (QAT) - REQUIRED FOR ai8x-synthesis
"""
Enable QAT to automatically fold BatchNorm layers during training.
This is ESSENTIAL for ai8x-synthesis to work properly.

QAT will:
1. Fold BatchNorm layers into convolution weights
2. Quantize weights to 8-bit during training
3. Learn quantization-aware parameters
4. Make the model ready for MAX78002 deployment

Without QAT, the model will fail during synthesis with:
"ERROR: The checkpoint file contains 1-dimensional weights for `conv1_1.bn.weight`... 
Ensure the BatchNorm layers have been folded."
"""

print("="*80)
print("ENABLING QUANTIZATION-AWARE TRAINING (QAT)")
print("="*80)

# Define QAT policy (8-bit weights and bias)
qat_policy = {
    'weight_bits': 8,        # 8-bit weight quantization
    'bias_bits': 8,          # 8-bit bias quantization  
    'overrides': {}          # No layer-specific overrides
}

print("\nQAT Policy Configuration:")
print(f"  - Weight quantization: {qat_policy['weight_bits']} bits")
print(f"  - Bias quantization: {qat_policy['bias_bits']} bits")
print(f"  - Layer overrides: {len(qat_policy['overrides'])}")

# Initialize QAT - this prepares the model for quantization
# This will also fold BatchNorm layers automatically during training
ai8x.initiate_qat(model, qat_policy)

print("\n✓ QAT ENABLED SUCCESSFULLY")
print("  ✓ Model prepared for 8-bit quantization")
print("  ✓ BatchNorm layers will be fused automatically")
print("  ✓ Quantization-aware parameters initialized")
print("  ✓ Model ready for ai8x-synthesis after training")

# Verify QAT initialization
has_weight_bits = hasattr(list(model.modules())[1], 'weight_bits')
has_quantize = hasattr(list(model.modules())[1], 'quantize_weight')

print("\n✓ QAT Verification:")
print(f"  - weight_bits parameter: {'✓ Found' if has_weight_bits else '✗ Missing'}")
print(f"  - quantize_weight function: {'✓ Found' if has_quantize else '✗ Missing'}")

if has_weight_bits and has_quantize:
    print("\n🎉 QAT is properly configured!")
else:
    print("\n⚠️  Warning: QAT may not be fully initialized")

print("="*80)
print("\nNOTE: The checkpoint saved during training will now be QAT-compatible")
print("      and ready for quantization without additional BatchNorm folding.")
print("="*80)

In [ ]:
# Cell 11: Main Training Loop (ai8x-compatible)
"""
Training loop with:
- Compression scheduler integration
- Proper checkpointing for ai8x synthesis
- Learning rate reduction on plateau
- Early stopping
- History tracking
"""

# Training configuration
num_epochs = 100
best_val_loss = float('inf')
best_val_top1 = 0
patience = 15
patience_counter = 0

# History tracking
history = {
    'train_loss': [], 'train_top1': [], 'train_top5': [],
    'val_loss': [], 'val_top1': [], 'val_top5': [],
    'lr': []
}

# Learning rate scheduler (standard for ai8x training)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    factor=0.5, 
    patience=7, 
    verbose=True,
    min_lr=1e-6
)

print("="*80)
print("Starting ai8x-compatible training...")
print("="*80)
print(f"Device: {device}")
print(f"Total epochs: {num_epochs}")
print(f"Batch size: 64")
print(f"Initial LR: {optimizer.param_groups[0]['lr']}")
print(f"Early stopping patience: {patience}")
print("="*80)

# Main training loop
for epoch in range(num_epochs):
    # Train for one epoch using ai8x-compatible function
    train_loss, train_top1, train_top5 = train_epoch(
        model=model,
        dataloader=train_loader_split,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epoch=epoch,
        compression_scheduler=compression_scheduler,
        logger=print_logger
    )
    
    # Validate using ai8x-compatible function
    val_loss, val_top1, val_top5 = validate(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device,
        epoch=epoch,
        logger=print_logger
    )
    
    # Learning rate scheduling
    lr_scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_top1'].append(train_top1)
    history['train_top5'].append(train_top5)
    history['val_loss'].append(val_loss)
    history['val_top1'].append(val_top1)
    history['val_top5'].append(val_top5)
    history['lr'].append(current_lr)
    
    # Print epoch summary
    print(f"\n{'='*80}")
    print(f"Epoch {epoch:3d}/{num_epochs-1} Summary:")
    print(f"  Train -> Loss: {train_loss:.4f} | Top1: {train_top1:.2f}% | Top5: {train_top5:.2f}%")
    print(f"  Val   -> Loss: {val_loss:.4f} | Top1: {val_top1:.2f}% | Top5: {val_top5:.2f}%")
    print(f"  LR: {current_lr:.6f}")
    
    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_top1 = val_top1
        patience_counter = 0
        
        # Save best model checkpoint (ai8x-compatible format)
        checkpoint = {
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_top1': best_val_top1,
            'best_loss': best_val_loss,
            'optimizer': optimizer.state_dict(),
            'compression_scheduler': compression_scheduler.state_dict() if compression_scheduler else None,
        }
        torch.save(checkpoint, 'best_ecg_model_ai8x.pth.tar')
        print("  ✓ [BEST MODEL SAVED]")
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{patience})")
        
        if patience_counter >= patience:
            print(f"\n{'='*80}")
            print(f"Early stopping triggered! No improvement for {patience} epochs.")
            print(f"Best validation Top1: {best_val_top1:.2f}%")
            print(f"Best validation loss: {best_val_loss:.4f}")
            print(f"{'='*80}")
            break
    
    print(f"{'='*80}\n")

print("\n" + "="*80)
print("Training complete!")
print(f"Best validation Top1: {best_val_top1:.2f}%")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total epochs trained: {epoch + 1}")
print("="*80)

# Load best model
checkpoint = torch.load('best_ecg_model_ai8x.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
print("\n✓ Best model loaded from checkpoint")
print(f"  - Epoch: {checkpoint['epoch']}")
print(f"  - Top1 Accuracy: {checkpoint['best_top1']:.2f}%")
print(f"  - Validation Loss: {checkpoint['best_loss']:.4f}")


## Training Results Analysis (No SMOTE)

**Best Performance:**
- Validation Accuracy: 83.90% (Epoch 15)
- Without using synthetic SMOTE data - only real ECG signals

**Key Observations:**
1. ✓ **Normal beat detection should work well** - The majority class has plenty of samples
2. ⚠️ **High validation instability** - Large swings suggest overfitting to minority classes
3. ⚠️ **Class weights may be too aggressive** - Causing the model to overfit rare patterns

**Next Steps to Improve:**
1. Try **reducing class weight strength** (use `class_weight` parameter with smaller values)
2. Add **data augmentation** (time shifts, amplitude scaling) for minority classes
3. Use **focal loss** instead of weighted CrossEntropy for better handling of hard examples
4. Consider **ensemble methods** or **two-stage classification** (normal vs abnormal, then classify abnormal types)

Let's evaluate the test set to see per-class performance!

In [ ]:
# Cell 12: Plot Training History
"""
Visualize training progress
"""

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss plot
axes[0, 0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Top-1 Accuracy plot
axes[0, 1].plot(history['train_top1'], label='Train Top1', linewidth=2)
axes[0, 1].plot(history['val_top1'], label='Val Top1', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Top-1 Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Top-5 Accuracy plot
axes[1, 0].plot(history['train_top5'], label='Train Top5', linewidth=2)
axes[1, 0].plot(history['val_top5'], label='Val Top5', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Accuracy (%)', fontsize=12)
axes[1, 0].set_title('Top-5 Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Learning rate plot
axes[1, 1].plot(history['lr'], linewidth=2, color='red')
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Learning Rate', fontsize=12)
axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("Training Summary:")
print(f"  Final Train Top1: {history['train_top1'][-1]:.2f}%")
print(f"  Final Val Top1: {history['val_top1'][-1]:.2f}%")
print(f"  Best Val Top1: {max(history['val_top1']):.2f}%")
print(f"  Final LR: {history['lr'][-1]:.6f}")
print("="*80)


In [ ]:
# Cell 13: Final Test Evaluation
"""
Evaluate the best model on the test set
"""

print("="*80)
print("Evaluating best model on test set...")
print("="*80)

# Evaluate on test set
test_loss, test_top1, test_top5 = validate(
    model=model,
    dataloader=test_loader,
    criterion=criterion,
    device=device,
    epoch=-1,  # -1 indicates test evaluation
    logger=None
)

print("\n" + "="*80)
print("FINAL TEST RESULTS:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Top-1 Accuracy: {test_top1:.2f}%")
print(f"  Test Top-5 Accuracy: {test_top5:.2f}%")
print("="*80)

# Get detailed predictions for confusion matrix
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in tqdm(test_loader, desc='Generating predictions'):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.numpy())

all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

# Classification report
class_names_list = ['Normal (N)', 'Supraventricular (S)', 'Ventricular (V)', 'Fusion (F)', 'Unknown (Q)']
print("\n" + "="*80)
print("CLASSIFICATION REPORT:")
print("="*80)
print(classification_report(all_targets, all_preds, target_names=class_names_list, digits=4))

# Confusion matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names_list, 
            yticklabels=class_names_list)
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

print("\n✓ Model is ready for ai8x-synthesis!")
print("  - Use 'best_ecg_model_ai8x.pth.tar' for quantization")
print("  - Compatible with MAX78002 deployment")


In [ ]:
# Cell 14: Confusion Matrix Visualization & Analysis
"""
Detailed confusion matrix heatmap with per-class analysis
"""
import matplotlib.pyplot as plt
import seaborn as sns

# The confusion matrix should already exist from Cell 13
# If not, let's recreate it
if 'cm' not in locals() or 'all_preds' not in locals():
    print("Re-generating confusion matrix data...")
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc='Generating predictions'):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.numpy())
    
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    cm = confusion_matrix(all_targets, all_preds)

# Plot confusion matrix
class_names_list = ['Normal (N)', 'Supraventricular (S)', 'Ventricular (V)', 'Fusion (F)', 'Unknown (Q)']

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names_list, 
            yticklabels=class_names_list,
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - Test Set\n(83.90% Val Accuracy Model)', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=13, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print detailed analysis
print("\n" + "="*80)
print("CONFUSION MATRIX ANALYSIS:")
print("="*80)
print(f"\nTotal test samples: {cm.sum():,}")
print(f"\nPer-class breakdown:")
for i, class_name in enumerate(class_names_list):
    total = cm[i].sum()
    correct = cm[i, i]
    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"  {class_name:20s}: {correct:6,}/{total:6,} correct ({accuracy:5.2f}%)")

# Calculate where Normal beats are misclassified
print("\n" + "="*80)
print("NORMAL BEAT MISCLASSIFICATION ANALYSIS:")
print("="*80)
if cm.shape[0] > 0:
    normal_total = cm[0].sum()
    normal_correct = cm[0, 0]
    normal_acc = (normal_correct / normal_total * 100) if normal_total > 0 else 0
    
    print(f"Normal beats correctly classified: {normal_correct:,}/{normal_total:,} ({normal_acc:.2f}%)")
    print(f"\nMisclassified as:")
    for i in range(1, len(class_names_list)):
        misclass_count = cm[0, i]
        misclass_pct = (misclass_count / normal_total * 100) if normal_total > 0 else 0
        if misclass_count > 0:
            print(f"  → {class_names_list[i]:20s}: {misclass_count:6,} ({misclass_pct:5.2f}%)")

print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)

# Analyze each class performance
for i, class_name in enumerate(class_names_list):
    total = cm[i].sum()
    correct = cm[i, i]
    accuracy = (correct / total * 100) if total > 0 else 0
    
    if i == 0:  # Normal
        if accuracy < 70:
            print(f"🔴 {class_name}: {accuracy:.2f}% - CRITICAL: Normal beat detection is TOO LOW!")
        elif accuracy < 85:
            print(f"⚠️  {class_name}: {accuracy:.2f}% - Normal detection needs improvement")
        else:
            print(f"✓ {class_name}: {accuracy:.2f}% - Good normal beat detection")
    else:
        if accuracy < 10:
            print(f"⚠️  {class_name}: {accuracy:.2f}% - Extremely poor (class too rare or weights too aggressive)")
        elif accuracy < 50:
            print(f"⚠️  {class_name}: {accuracy:.2f}% - Poor performance")
        elif accuracy < 80:
            print(f"→ {class_name}: {accuracy:.2f}% - Moderate performance")
        else:
            print(f"✓ {class_name}: {accuracy:.2f}% - Good performance")

print("="*80)

## 🎯 Baseline Results Analysis (No Class Weights)

### ✅ Excellent Performance Where It Matters!

**Critical Classes (Medical Priority):**
- **Normal beats: 89.81%** ✓ Excellent! This is clinically useful
- **Ventricular: 72.36%** ✓ Good! The most dangerous arrhythmia is being detected

**Rare Classes (Expected Poor Performance):**
- Supraventricular: 0.98% - Too rare to learn naturally (~1.5% of data)
- Fusion: 0.00% - Too rare (~0.3% of data)  
- Unknown: 0.00% - Only 7 test samples total

### 📊 Comparison: Baseline vs Class Weights

| Class | No Weights (Baseline) | With Aggressive Weights | Winner |
|-------|----------------------|-------------------------|---------|
| **Normal** | **89.81%** | 56.55% | ✅ Baseline (+33%) |
| **Ventricular** | **72.36%** | 80.56% | ⚠️ Weights (+8%) |
| Supraventricular | 0.98% | 6.75% | Weights (+6%) |
| Fusion | 0.00% | 6.19% | Weights (+6%) |
| Unknown | 0.00% | 14.29% | Weights (+14%) |

**Verdict:** The baseline is **FAR superior** for clinical use:
- ✅ 89.81% normal detection vs 56.55% (unacceptable)
- ✅ Stable training (no wild swings)
- ✅ 72% ventricular detection is clinically useful
- The rare classes perform poorly either way (0-14%), so this is acceptable

### 🏥 Clinical Interpretation

For a wearable ECG monitor (MAX78002), this model is **actually quite good**:

1. **Normal beat detection (89.81%)** - Most important metric. Nearly 9 in 10 normal beats detected correctly
2. **Ventricular detection (72.36%)** - The most life-threatening arrhythmia. 7 in 10 detected
3. **Low false positives** - Only 10% of normals misclassified (7.37% as ventricular, 2.29% as fusion)

### 🎯 Recommended Next Steps

**Option 1: Ship This Baseline (Recommended)**
- This model is clinically useful as-is
- Deploy to MAX78002 and collect real-world data
- Focus on Normal + Ventricular only (ignore rare classes)

**Option 2: Improve Ventricular Detection (Target 80%+)**
- Add mild class weighting (weights * 0.3) to boost ventricular without hurting normal
- Or train a **2-class model**: Normal vs Ventricular only

**Option 3: Two-Stage Classifier**
- Stage 1: Normal vs Abnormal (should get ~95%)  
- Stage 2: If abnormal, classify type (ventricular/other)

**NOT Recommended:**
- ❌ Aggressive class weights (destroys normal detection)
- ❌ SMOTE (creates unrealistic synthetic ECG signals)
- ❌ Trying to detect fusion/unknown (too rare, medically less critical)

### 💡 Key Insight

**For medical devices, getting the common patterns right is more important than detecting every rare pattern.** A model that catches 90% of normal beats and 72% of ventricular beats will save more lives than one that catches 56% of normal beats but 80% of everything else.

# Generate KAT (Known Answer Test) Sample

Before synthesis, we need to generate a KAT (Known Answer Test) sample. This is a test input that ai8x-synthesis will use to verify the hardware implementation produces the same outputs as the software model.

In [ ]:
# Generate KAT sample from test set
# KAT = Known Answer Test - used by ai8x-synthesis to verify hardware implementation

# Get one sample from test loader
x, y = next(iter(test_loader))

# Convert to int8 format (ai8x uses int8 quantization)
# Scale from [-1, 1] to [-128, 127] range
kat_sample = (x[0].numpy() * 128).astype(np.int64)
kat_sample = np.clip(kat_sample, -128, 127)

print(f"KAT sample shape: {kat_sample.shape}")
print(f"KAT sample range: [{kat_sample.min()}, {kat_sample.max()}]")
print(f"True label: {class_names[y[0].item()]}")

# Visualize the KAT sample
plt.figure(figsize=(12, 3))
plt.plot(kat_sample[0])  # Plot the single channel ECG signal
plt.title(f'KAT Sample - {class_names[y[0].item()]}', fontsize=14, fontweight='bold')
plt.xlabel('Time Sample', fontsize=12)
plt.ylabel('Amplitude (int8)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Save KAT sample for synthesis (shape: 1, 128 for 1D ECG)
np.save('sample_ecg_1x128.npy', kat_sample, allow_pickle=False, fix_imports=False)
print("\n✓ KAT sample saved to: sample_ecg_1x128.npy")
print("  - This file is required for ai8x-synthesis")
print("  - Use it with --sample-input flag during synthesis")

# Verify Model Checkpoint Format

The checkpoint saved during training already contains all necessary metadata for ai8x-synthesis. Let's verify it has the correct structure.

In [ ]:
# Load and inspect the checkpoint
checkpoint_path = 'best_ecg_model_ai8x.pth.tar'
checkpoint = torch.load(checkpoint_path)

print("="*80)
print("CHECKPOINT CONTENTS:")
print("="*80)
print("\nAvailable keys:")
for key in checkpoint.keys():
    print(f"  ✓ {key}")

print("\n" + "="*80)
print("CHECKPOINT DETAILS:")
print("="*80)
print(f"Epoch: {checkpoint['epoch']}")
print(f"Best Top1 Accuracy: {checkpoint['best_top1']:.2f}%")
print(f"Best Loss: {checkpoint['best_loss']:.4f}")

if 'compression_scheduler' in checkpoint and checkpoint['compression_scheduler'] is not None:
    print(f"Compression Scheduler: ✓ Present")
else:
    print(f"Compression Scheduler: Not used (optional)")

print("\n" + "="*80)
print("MODEL STATE DICT:")
print("="*80)
print(f"Number of layers: {len(checkpoint['state_dict'])}")
print(f"\nFirst few layers:")
for i, (name, param) in enumerate(list(checkpoint['state_dict'].items())[:5]):
    print(f"  {name}: {tuple(param.shape)}")

print("\n" + "="*80)
print("SYNTHESIS READINESS:")
print("="*80)
print(f"✓ Checkpoint file: {checkpoint_path}")
print(f"✓ KAT sample file: sample_ecg_1x128.npy")
print(f"✓ Model architecture: ECGClassifier (ai8x-compatible)")
print(f"✓ Device target: MAX78002 (device 87)")
print("\n" + "="*80)
print("READY FOR ai8x-synthesis!")
print("="*80)
print("\nNext step: Use ai8x-synthesis tool to generate C code for MAX78002")
print("Example command:")
print("  python ai8x-synthesis/network_generator.py \\")
print("    --checkpoint-file best_ecg_model_ai8x.pth.tar \\")
print("    --sample-input sample_ecg_1x128.npy \\")
print("    --device MAX78002 \\")
print("    --compact-data \\")
print("    --verbose")

In [ ]:
# Cell: Verify BatchNorm Folding for ai8x-synthesis
"""
Verify that the checkpoint is ready for ai8x-synthesis by checking:
1. No BatchNorm layers remain in the state_dict
2. All weights are properly quantized
3. Model is compatible with MAX78002 synthesis
"""

print("="*80)
print("VERIFYING CHECKPOINT FOR ai8x-synthesis")
print("="*80)

# Load the saved checkpoint
checkpoint_path = 'best_ecg_model_ai8x.pth.tar'
checkpoint = torch.load(checkpoint_path)

print(f"\n✓ Checkpoint loaded: {checkpoint_path}")
print(f"  - Epoch: {checkpoint['epoch']}")
print(f"  - Best Top1: {checkpoint['best_top1']:.2f}%")
print(f"  - Best Loss: {checkpoint['best_loss']:.4f}")

# Check for BatchNorm layers in state_dict
state_dict_keys = list(checkpoint['state_dict'].keys())
bn_keys = [k for k in state_dict_keys if '.bn.' in k]

print(f"\n{'='*80}")
print("BATCHNORM FOLDING VERIFICATION")
print(f"{'='*80}")

if bn_keys:
    print("❌ ERROR: BatchNorm layers NOT folded!")
    print(f"   Found {len(bn_keys)} BatchNorm keys:")
    for key in bn_keys[:10]:  # Show first 10
        print(f"   - {key}")
    if len(bn_keys) > 10:
        print(f"   ... and {len(bn_keys) - 10} more")
    print("\n⚠️  This checkpoint will FAIL during ai8x-synthesis!")
    print("   Action: Re-run training with QAT enabled (Cell 10.5)")
else:
    print("✅ SUCCESS: No BatchNorm layers found!")
    print("   All BatchNorm layers have been properly fused")
    print("   This checkpoint is ready for ai8x-synthesis")

# Check for QAT parameters
qat_keys = [k for k in state_dict_keys if 'weight_bits' in k or 'bias_bits' in k or 'output_shift' in k]

print(f"\n{'='*80}")
print("QAT PARAMETERS VERIFICATION")
print(f"{'='*80}")

if qat_keys:
    print(f"✅ SUCCESS: Found {len(qat_keys)} QAT parameters")
    print("   Examples:")
    for key in qat_keys[:5]:  # Show first 5
        print(f"   - {key}")
    if len(qat_keys) > 5:
        print(f"   ... and {len(qat_keys) - 5} more")
    print("\n   QAT was properly applied during training")
else:
    print("❌ WARNING: No QAT parameters found")
    print("   The model may not have been trained with QAT enabled")
    print("   Action: Re-run training with QAT enabled (Cell 10.5)")

# Summary
print(f"\n{'='*80}")
print("CHECKPOINT COMPATIBILITY SUMMARY")
print(f"{'='*80}")

if not bn_keys and qat_keys:
    print("🎉 CHECKPOINT IS READY FOR ai8x-synthesis!")
    print("\nNext steps:")
    print("1. Copy checkpoint to ai8x-synthesis directory")
    print("2. Run quantization: python3 quantize.py \\")
    print("                     best_ecg_model_ai8x.pth.tar \\")
    print("                     best_ecg_model_ai8x_q8.pth.tar \\")
    print("                     --device MAX78002 -v")
    print("3. Run synthesis: python3 ai8xize.py \\")
    print("                  --checkpoint-file best_ecg_model_ai8x_q8.pth.tar \\")
    print("                  --config-file networks/ecg-net.yaml \\")
    print("                  --device MAX78002 ...")
elif not bn_keys:
    print("⚠️  PARTIAL SUCCESS: No BatchNorm layers found")
    print("   However, QAT parameters are missing")
    print("   The checkpoint may work but is not optimal")
    print("\n   Recommended: Re-run training with QAT enabled")
else:
    print("❌ CHECKPOINT NOT READY")
    print("   BatchNorm layers must be folded before synthesis")
    print("\n   REQUIRED ACTION: Re-run training with QAT enabled (Cell 10.5)")

print(f"{'='*80}\n")

In [ ]:
# Cell: Fuse BatchNorm Layers for ai8x-synthesis
"""
Manually fuse BatchNorm layers into convolution layers.
This is REQUIRED for ai8x-synthesis to work.

QAT was enabled during training, but BatchNorm fusion needs to be done explicitly.
This cell will:
1. Load the checkpoint
2. Fuse all BatchNorm layers into their parent convolution layers
3. Save the checkpoint with fused layers
"""

print("="*80)
print("FUSING BATCHNORM LAYERS")
print("="*80)

# Load the checkpoint
checkpoint_path = 'best_ecg_model_ai8x.pth.tar'
checkpoint = torch.load(checkpoint_path)

print(f"\n✓ Checkpoint loaded: {checkpoint_path}")
print(f"  - Epoch: {checkpoint['epoch']}")
print(f"  - Best Top1: {checkpoint['best_top1']:.2f}%")
print(f"  - Best Loss: {checkpoint['best_loss']:.4f}")

# Load state dict into model
model.load_state_dict(checkpoint['state_dict'])
print("\n✓ Model state loaded")

# Count BatchNorm layers before fusion
bn_keys_before = [k for k in checkpoint['state_dict'].keys() if '.bn.' in k]
print(f"  - BatchNorm keys before fusion: {len(bn_keys_before)}")

# Fuse BatchNorm layers
print("\nFusing BatchNorm layers into convolution weights...")
ai8x.fuse_bn_layers(model)

print("✓ BatchNorm layers fused!")

# Get new state dict
fused_state_dict = model.state_dict()

# Count BatchNorm layers after fusion
bn_keys_after = [k for k in fused_state_dict.keys() if '.bn.' in k]
print(f"  - BatchNorm keys after fusion: {len(bn_keys_after)}")

if len(bn_keys_after) == 0:
    print("\n✅ SUCCESS: All BatchNorm layers have been fused!")
else:
    print(f"\n⚠️  WARNING: Still found {len(bn_keys_after)} BatchNorm keys")

# Update checkpoint with fused state dict
checkpoint['state_dict'] = fused_state_dict

# Save the fused checkpoint
fused_checkpoint_path = 'best_ecg_model_ai8x.pth.tar'
torch.save(checkpoint, fused_checkpoint_path)

print(f"\n✓ Checkpoint saved with fused layers: {fused_checkpoint_path}")

print("\n" + "="*80)
print("VERIFICATION")
print("="*80)

# Verify the saved checkpoint
verify_checkpoint = torch.load(fused_checkpoint_path)
verify_bn_keys = [k for k in verify_checkpoint['state_dict'].keys() if '.bn.' in k]
verify_qat_keys = [k for k in verify_checkpoint['state_dict'].keys() if 'weight_bits' in k or 'bias_bits' in k]

print(f"Saved checkpoint verification:")
print(f"  - BatchNorm keys: {len(verify_bn_keys)}")
print(f"  - QAT parameter keys: {len(verify_qat_keys)}")

if len(verify_bn_keys) == 0 and len(verify_qat_keys) > 0:
    print("\n🎉 CHECKPOINT IS NOW READY FOR ai8x-synthesis!")
    print("\n" + "="*80)
    print("NEXT STEPS:")
    print("="*80)
    print("\n1. Copy files to ai8x-synthesis directory:")
    print("   cp best_ecg_model_ai8x.pth.tar ~/ai8x-synthesis/")
    print("   cp sample_ecg_1x128.npy ~/ai8x-synthesis/")
    print("   cp networks/ecg-net.yaml ~/ai8x-synthesis/networks/")
    print("\n2. Run quantization:")
    print("   cd ~/ai8x-synthesis")
    print("   python3 quantize.py best_ecg_model_ai8x.pth.tar \\")
    print("                       best_ecg_model_ai8x_q8.pth.tar \\")
    print("                       --device MAX78002 -v")
    print("\n3. Run synthesis:")
    print("   python3 ai8xize.py --test-dir sdk/Examples/MAX78002/CNN \\")
    print("                      --prefix ecg_classifier \\")
    print("                      --checkpoint-file best_ecg_model_ai8x_q8.pth.tar \\")
    print("                      --config-file networks/ecg-net.yaml \\")
    print("                      --device MAX78002 \\")
    print("                      --compact-data --mexpress --timer 0 \\")
    print("                      --display-checkpoint --verbose --overwrite")
else:
    print("\n❌ ERROR: Checkpoint still not ready")
    print(f"   BatchNorm keys: {len(verify_bn_keys)} (should be 0)")
    print(f"   QAT keys: {len(verify_qat_keys)} (should be > 0)")

print("="*80)
